# Apply ICE CREAMS Model to Labelled Validation

In [ ]:
from fastai.tabular.all import load_learner
import pandas as pd

This script uses a large labelled dataset created to validate ICE CREAMS. It consists of a selection of labelled sentinel-2 pixels from across Europe and across the classes the ICE CREAMS predicts. This script focuses on Magniolopsida (Seagrass) and has a cut off of 0.246 NDVI (equivalent to 20 % Seagrass cover). First we load the pickled model, then we load the labelled validation data. We apply the model to the labelled data and then compare the the ratios and numbers of True Positive (TP), True Negative (TN), False Positive (FP) and False Negative (FN). These are then used to calculate accuracy and F1 scores. 

In [ ]:
learn = load_learner('Models/ICECREAMS_V1_1.pkl')

In [ ]:
df_test = pd.read_csv('Data/Input/Validation/Validation_Europe_LabelledS2.csv', low_memory=False,sep=",")

In [ ]:
df_test

In [ ]:
caselist=[(df_test.Label_Char!="Magnoliopsida", 'Absence'), 
          ((df_test.Label_Char=="Magnoliopsida") & (df_test.NDVI.lt(0.246)), 'Absence'), 
          (df_test.Label_Char=="Magnoliopsida", 'Presence')]

df_test=df_test.assign(Label_Char_PvA = df_test.Label_Char.case_when(caselist=caselist))

In [ ]:
dl = learn.dls.test_dl(df_test, bs=4000)
preds,_ = learn.get_preds(dl=dl)

In [ ]:
class_idxs = preds.argmax(axis=1)
res = [learn.dls.vocab[c] for c in class_idxs]

In [ ]:
class_probs= preds.max(axis=1)
class_probs=class_probs.values

In [ ]:
NumPred= class_idxs.tolist()

In [ ]:
PredProbs =class_probs.tolist()

In [ ]:
res_df= pd.DataFrame(list(zip(df_test['x'],
                              df_test['y'],
                              df_test['Label_Char_PvA'],
                              df_test['NDVI'],
                              res,
                              PredProbs)),
                     columns =['x','y','Label_Char_PvA','NDVI','Pred_Class','Prob'])
res_df

In [ ]:
res_df.groupby(['Label_Char_PvA','Pred_Class'])['x'].count()

In [ ]:
caselist=[(res_df.Pred_Class!="Magnoliopsida", 'Absence'), 
          ((res_df.Pred_Class=="Magnoliopsida") & (res_df.NDVI.lt(0.246)), 'Absence'), 
          (res_df.Pred_Class=="Magnoliopsida", 'Presence')]
res_df=res_df.assign(Pred_Class_PvA = res_df.Pred_Class.case_when(caselist=caselist))

In [ ]:
Results=res_df.groupby(['Label_Char_PvA','Pred_Class_PvA'])['x'].count().reset_index()

In [ ]:
Results

In [ ]:
TP=Results.query("Label_Char_PvA == 'Presence' and Pred_Class_PvA == 'Presence'")[['x']].iloc[0]
TN=Results.query("Label_Char_PvA == 'Absence' and Pred_Class_PvA == 'Absence'")[['x']].iloc[0]
FP=Results.query("Label_Char_PvA == 'Absence' and Pred_Class_PvA == 'Presence'")[['x']].iloc[0]
FN=Results.query("Label_Char_PvA == 'Presence' and Pred_Class_PvA == 'Absence'")[['x']].iloc[0]
Precision=TP/(TP+FP)
Recall=TP/(TP+FN)
F1=(2*Precision*Recall)/(Precision+Recall)

In [ ]:
Accuracy=((TP+TN)/(TP+TN+FN+FP))*100

In [ ]:
Accuracy

In [ ]:
F1

In [ ]:
Precision

In [ ]:
Recall